<a href="https://colab.research.google.com/github/larissagarcia/Agent-IA-csv/blob/main/Agent_jjj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# Célula 1
!pip install -q streamlit pandas numpy matplotlib plotly scikit-learn sqlalchemy openai fpdf python-dotenv pyngrok
# instalar node/npm para localtunnel (opcional)
!apt-get update -qq && apt-get install -y -qq nodejs npm

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [57]:
%%bash
# Célula 2: cria arquivos do projeto
mkdir -p /content/project

cat > /content/project/eda.py <<'PY'
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

def summary_stats(df):
    return df.describe(include='all', datetime_is_numeric=True).T

def hist_plot(df, column, bins=50):
    fig, ax = plt.subplots()
    df[column].dropna().hist(bins=bins, ax=ax)
    ax.set_title(f"Histograma: {column}")
    return fig

def corr_matrix(df):
    return df.corr()

def detect_outliers_isolationforest(df, numeric_cols, contamination=0.01):
    if not numeric_cols:
        return df.iloc[0:0]
    model = IsolationForest(contamination=contamination, random_state=42)
    X = df[numeric_cols].fillna(0)
    preds = model.fit_predict(X)
    return df[preds == -1]
PY

cat > /content/project/memory.py <<'PY'
import sqlite3, json
from datetime import datetime

class Memory:
    def __init__(self, db_path="/content/project/memory.sqlite"):
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self._create_tables()
    def _create_tables(self):
        c = self.conn.cursor()
        c.execute("""CREATE TABLE IF NOT EXISTS interactions (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        timestamp TEXT,
                        question TEXT,
                        answer TEXT,
                        meta TEXT
                    )""")
        self.conn.commit()
    def add_interaction(self, question, answer, meta=None):
        c = self.conn.cursor()
        c.execute("INSERT INTO interactions (timestamp, question, answer, meta) VALUES (?, ?, ?, ?)",
                  (datetime.utcnow().isoformat(), question, answer, json.dumps(meta or {})))
        self.conn.commit()
    def get_all(self, limit=50):
        c = self.conn.cursor()
        c.execute("SELECT timestamp, question, answer, meta FROM interactions ORDER BY id DESC LIMIT ?", (limit,))
        return c.fetchall()
PY

cat > /content/project/agent.py <<'PY'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from eda import summary_stats, hist_plot, corr_matrix, detect_outliers_isolationforest
from sklearn.cluster import KMeans

class Agent:
    def __init__(self, df, memory):
        self.df = df.copy()
        self.memory = memory
        self.numeric_cols = self.df.select_dtypes(include=['number']).columns.tolist()
        self.categorical_cols = self.df.select_dtypes(exclude=['number']).columns.tolist()

    def handle_question(self, question: str):
        q = question.lower()

        # --- 1) Descrição dos Dados ---
        if "tipo" in q or "dados" in q:
            text = f"Colunas numéricas: {self.numeric_cols}\nColunas categóricas: {self.categorical_cols}"
            self.memory.add_interaction(question, text)
            return {"text": text}

        if "distribui" in q or "histograma" in q:
            figs = []
            for col in self.numeric_cols[:6]:
                figs.append(hist_plot(self.df, col))
            text = "Distribuição (histogramas) das principais variáveis numéricas."
            self.memory.add_interaction(question, text)
            return {"text": text, "fig": figs[0]}

        if "intervalo" in q or "mínimo" in q or "máximo" in q:
            stats = self.df[self.numeric_cols].agg(['min','max']).T
            text = "Intervalos (mínimo e máximo) das variáveis numéricas."
            self.memory.add_interaction(question, text)
            return {"text": text, "table": stats}

        if "média" in q or "mediana" in q or "tendência central" in q:
            stats = self.df[self.numeric_cols].agg(['mean','median']).T
            text = "Médias e medianas das variáveis numéricas."
            self.memory.add_interaction(question, text)
            return {"text": text, "table": stats}

        if "variabilidade" in q or "desvio padrão" in q or "variância" in q:
            stats = self.df[self.numeric_cols].agg(['std','var']).T
            text = "Medidas de variabilidade (desvio padrão e variância)."
            self.memory.add_interaction(question, text)
            return {"text": text, "table": stats}

        # --- 2) Padrões e Tendências ---
        if "tendência" in q or "temporal" in q:
            time_cols = [c for c in self.df.columns if "time" in c.lower() or "date" in c.lower()]
            if time_cols:
                col = time_cols[0]
                ts = self.df.groupby(col).size()
                fig, ax = plt.subplots()
                ts.plot(ax=ax)
                ax.set_title(f"Tendência temporal por {col}")
                text = f"Tendência temporal detectada pela coluna {col}."
                self.memory.add_interaction(question, text)
                return {"text": text, "fig": fig}
            else:
                text = "Não foi encontrada coluna temporal explícita."
                self.memory.add_interaction(question, text)
                return {"text": text}

        if "frequente" in q or "menos frequente" in q or "moda" in q:
            freq = {}
            for col in self.categorical_cols[:5]:
                freq[col] = self.df[col].value_counts().head(5).to_dict()
            text = "Valores mais frequentes por variável categórica."
            self.memory.add_interaction(question, text)
            return {"text": text, "table": pd.DataFrame(freq)}

        if "cluster" in q or "agrup" in q:
            if len(self.numeric_cols) >= 2:
                km = KMeans(n_clusters=3, random_state=42).fit(self.df[self.numeric_cols].fillna(0))
                df2 = self.df.copy()
                df2["_cluster"] = km.labels_
                text = "Clusterização aplicada (k=3)."
                self.memory.add_interaction(question, text)
                return {"text": text, "table": df2["_cluster"].value_counts().reset_index().rename(columns={'index':'cluster','_cluster':'count'})}
            else:
                return {"text":"Poucas colunas numéricas para aplicar clusters."}

        # --- 3) Detecção de Outliers ---
        if "outlier" in q or "atípico" in q or "anomalia" in q:
            out = detect_outliers_isolationforest(self.df, self.numeric_cols)
            text = f"Foram detectados {len(out)} outliers usando IsolationForest."
            self.memory.add_interaction(question, text)
            return {"text": text, "table": out.head(50)}

        # --- 4) Relações entre Variáveis ---
        if "correlação" in q or "relacion" in q:
            corr = corr_matrix(self.df)
            text = "Matriz de correlação entre variáveis numéricas."
            self.memory.add_interaction(question, text)
            return {"text": text, "table": corr}

        if "dispersão" in q or "scatter" in q:
            if len(self.numeric_cols) >= 2:
                x, y = self.numeric_cols[:2]
                fig, ax = plt.subplots()
                ax.scatter(self.df[x], self.df[y], alpha=0.3)
                ax.set_xlabel(x); ax.set_ylabel(y); ax.set_title(f"Dispersão entre {x} e {y}")
                text = f"Gráfico de dispersão entre {x} e {y}."
                self.memory.add_interaction(question, text)
                return {"text": text, "fig": fig}
            else:
                return {"text":"Não há colunas numéricas suficientes para scatter plot."}

        # --- fallback ---
        text = "Não reconheci a pergunta. Exemplos: tipos de dados, distribuições, intervalos, médias, variabilidade, tendências, frequências, clusters, outliers, correlação, dispersão."
        self.memory.add_interaction(question, text)
        return {"text": text}
PY

cat > /content/project/report_generator.py <<'PY'
from fpdf import FPDF

def generate_pdf_report(memory, output_path="/content/project/Agentes_Autonomos_Relatorio.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", "B", 14)
    pdf.cell(0,10, "Relatório - Agente EDA", ln=True, align="C")
    pdf.ln(6)
    pdf.set_font("Arial", size=11)
    interactions = memory.get_all(limit=6)
    if not interactions:
        pdf.multi_cell(0,6, "Sem interações.")
    for ts, q, a, meta in interactions:
        pdf.multi_cell(0,6, f"Pergunta: {q}")
        pdf.multi_cell(0,6, f"Resposta (resumo): {a}")
        pdf.ln(2)
    pdf.output(output_path)
    return output_path
PY

cat > /content/project/app.py <<'PY'
import streamlit as st
import pandas as pd
from memory import Memory
from agent import Agent
from report_generator import generate_pdf_report

st.set_page_config(page_title="Agente EDA (Colab)", layout="wide")
st.title("Agente E.D.A. — Colab")

uploaded = st.file_uploader("Carregue um CSV", type=["csv","zip"])
if uploaded is None:
    st.info("Envie um CSV para começar (ex.: creditcardfraud.csv).")
    st.stop()

# lê CSV
import io
df = pd.read_csv(io.BytesIO(uploaded.read()))
st.write(f"Linhas: {df.shape[0]} — Colunas: {df.shape[1]}")
st.dataframe(df.head())

# init
mem = Memory("/content/project/memory.sqlite")
agent = Agent(df, mem)

q = st.text_input("Pergunta sobre o dataset (ex.: 'Mostre histograma da coluna Amount')")
if st.button("Enviar pergunta"):
    if not q.strip():
        st.warning("Digite uma pergunta.")
    else:
        with st.spinner("Processando..."):
            resp = agent.handle_question(q)
        st.markdown("### Resposta do agente")
        st.write(resp.get("text"))
        if "table" in resp:
            st.dataframe(resp["table"])
        if "fig" in resp:
            st.pyplot(resp["fig"])

if st.button("Gerar relatório PDF"):
    pdf_path = generate_pdf_report(mem, output_path="/content/project/Agentes_Autonomos_Relatorio.pdf")
    with open(pdf_path, "rb") as f:
        st.download_button("Download Relatório PDF", f, file_name="Agentes_Autonomos_Relatorio.pdf")
PY

In [58]:
# Célula 3 (opcional)
from google.colab import drive
drive.mount('/content/drive')
# recomenda: criar /content/drive/MyDrive/eda_agent_project e copiar arquivos lá para persistência


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
# Célula 4 (opcional, insira apenas se for usar LLM)
from getpass import getpass
import os
key = getpass("Cole sua OPENAI_API_KEY (entrada oculta): ")
os.environ["OPENAI_API_KEY"] = key
os.environ["AGENT_MODE"] = "rule"  # por padrão 'rule'; altere p/ 'llm' se quiser ativar LLM
print("Variáveis configuradas (AGENT_MODE está em rule por padrão).")


Cole sua OPENAI_API_KEY (entrada oculta): ··········
Variáveis configuradas (AGENT_MODE está em rule por padrão).


In [60]:
# Célula 5A
import os, time, subprocess, sys
get_ipython().system_raw('streamlit run /content/project/app.py --server.port 8501 &')
print("Streamlit iniciado em background na porta 8501.")
time.sleep(1)


Streamlit iniciado em background na porta 8501.


In [61]:
!pip install pyngrok

In [66]:
from getpass import getpass
from pyngrok import ngrok

# pede para você digitar o token (não aparece na tela)
authtoken = getpass("Cole aqui seu ngrok authtoken (entrada oculta): ")

# configura o token
ngrok.set_auth_token(authtoken)
print("✅ Token configurado com sucesso!")

Cole aqui seu ngrok authtoken (entrada oculta): ··········
✅ Token configurado com sucesso!


In [68]:
# roda o Streamlit em background
get_ipython().system_raw('streamlit run /content/project/app.py --server.port 8501 &')

# cria o túnel com ngrok
public_url = ngrok.connect(8501)
print("🌍 URL pública:", public_url)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 5 endpoints over a single ngrok agent session.\nThe endpoints already running on this session are:\ntn_33Re2dWZ3rt1MiCwyti2Mh8J0j7, tn_33ReBAy1yFN7IiK6yBhA9uOWcLV, tn_33ReyXLxJrNSuR2VWSpaWa9E8is, tn_33RbZ1eqBF5edWOibrnJHCej3vv, tn_33RbeSnRP6UjWgGtyqdZw9Ade43.\nUpgrade to a Pay-as-you-go plan at: https://dashboard.ngrok.com/billing/choose-a-plan?plan=paygo\r\n\r\nERR_NGROK_324\r\n"}}


In [ ]:
from google.colab import files
files.download('/content/project/Agentes_Autonomos_Relatorio.pdf')


In [ ]:
# Célula 7 - parar streamlit
!pkill -f streamlit
# remover chave da sessão
import os
if "OPENAI_API_KEY" in os.environ:
    del os.environ["OPENAI_API_KEY"]
print("Streamlit parado e chave removida (se existia).")